imports

In [2]:
import numpy as np
from sympy import sqrt, sin, symbols, Function
from IPython.display import display
import operator
import functools

SDE

In [73]:
# F = ma = sum of:
    # -friction [gamma]
    # noise [xi]
    # noise intensity [D]
    # potential(x) [U]
    # oscillating force amplitude [a]
    # oscillating force(omega, t)
    # constant force [f]

class Sde():
    
    @staticmethod
    def get_default_options():
        return {
            'simulation':{
                'spp':100,         # steps per period
                'periods':2,#000,    # number of periods in the simulation
                'paths':2,#56,       # number of paths to sample
                'samples':100,     # sample the position every N steps
                'transients_number':200,      # number of periods to ignore
                'transients_fraction':0.1,    # fraction of periods to ignore
                'transients_type':'fraction', # periods to ignore because of transients (fraction, number)
                'rng_seed':None,
                'precision':'single',         # precision of the floating-point numbers (single, double)
                'rng_generator':'kiss32', 
                'deterministic':False,        # do not generate any noises
            },
            'output':{
                'mode':'summary',  # output mode (summary, path)
                'format':'text',   # output file format (text, npy)
                'destination':'./sde_out'
            },
            'gpu':{
                'cuda':True,
            },
            'debug':{
                'enabled':True,
            }
        }
        
    def __init__(self, variables, parameters, functions, equation, options = get_default_options.__func__()):
        self.variables = variables
        self.parameters = parameters
        self.functions = functions
        self.equation = equation
        self.options = options
        
        # parameters and variables in one dict
        self.params_vars = {**self.parameters, **self.variables}
        
        # name to object mapping
        # TODO is this necessary? could just use symbols()
        self.name_to_sympy = {}
        for p in self.params_vars:
            self.name_to_sympy[p.name] = p
        
        self._init_cuda()
    
    def _init_cuda(self):
        if self.options['gpu']['cuda']:
            import pycuda.driver as cuda
            cuda.init()
    
    def _validate(self):
        for symbol in self.equation.free_symbols:
            if symbol not in self.params_vars:
                raise ValueError('Value(s) for \'%s\' not provided.' % symbol.name)
        
        # dt
        if symbols('dt') not in self.params_vars and symbols('omega') in self.params_vars:
            self.name_to_sympy['dt'] = symbols('dt')
            self.parameters[self.name_to_sympy['dt']] = 2*np.pi/self.parameters[self.name_to_sympy['omega']]
        else:
            raise ValueError('Value is provided for neither \'dt\' nor \'omega\'.')
    
    def _simulate(self):
        if self.options['gpu']['cuda']:
            self._simulate_gpu()
        else:
            self._simulate_cpu()
            
    def _simulate_cpu(self):
        current_values = self.variables.copy()
        
        for path in range(self.options['simulation']['paths']):
            dt = self.parameters[symbols('dt')]
            
            for period in range(self.options['simulation']['periods']):
                for step in range(self.options['simulation']['spp']):
                    result = self.equation.evalf(subs = current_values)
                    current_values[self.name_to_sympy['t']] += dt
                    current_values[self.name_to_sympy['v']] += result * dt
                    current_values[self.name_to_sympy['x']] += current_values[self.name_to_sympy['v']] * dt
    
    def _prep_gpu(self):
        # TODO:
        # parametr o pojedynczej wartosci -> wygeneruj __constant__ na CUDA
        # parametr o liście wartości -> przekaż tablicę do global memory
        return
    
    def _simulate_gpu(self):
        _prep_gpu()
        return
    
    def start(self):
        self._validate()
        self._simulate()
        
    def indices_1D_ND(index, arrs):
        '''
        :param arrs:  array of arrays containing variables/parameters values
                      e.g.: [[0], [0,1,2,3,4,5,6], range(95,105), [2], [1], [0.1, 0.01, 0.001]]
        :param index: if all possible values combinations were produced from respectible subarrays and then flattened into 1D
                      this is gonna be an index of values set in such 1D array
        :returns:     array of indices for accessing desired values in arrs (indices in the same order as arrs)
        :raises ValueError: if index out of range
        '''
        indices_reversed = []
        lens = [len(x) for x in arrs]
        max_index = functools.reduce(operator.mul, lens, 1)
        if index < 0 or index >= max_index:
            raise ValueError('index >= {}'.format(max_index))
        for l in reversed(lens):
            indices_reversed.append(index%l)
            index = index // l
        return(indices_reversed[::-1])


usage

In [76]:
# variables with initial values
var_x, var_v, var_t = symbols('x v t')
# TODO these values should not be requested from user
variables = {
    var_x:0,
    var_v:1,
    var_t:0,
}

# parameters with values
par_gamma, par_amplitude, par_potential_aplitude, par_omega, par_const_force, par_noise_intensity \
    = symbols('gamma a Delta_U omega f D')
parameters = {
    par_gamma:1,
    par_amplitude:1,
    par_potential_aplitude:1,
    par_omega:1,
#     par_const_force:np.linspace(0, 1, 10),
    par_const_force:1,
    par_noise_intensity:1,
}

# functions
def oscillation():
    return sin

def potential():
    return sin

f_potential = Function('U')
f_oscillation = oscillation
f_noise = Function('xi')
functions = {f_potential, f_oscillation, f_noise}

# equation
equation = -par_gamma*var_v + par_potential_aplitude * f_potential(var_x) + par_amplitude * f_oscillation()(par_omega*var_t) \
    + par_const_force + sqrt(2*par_noise_intensity) * f_noise(var_t)
equation2 = -par_gamma*var_v + par_potential_aplitude * sin(var_x) + par_amplitude * sin(par_omega*var_t) \
    + par_const_force + sqrt(2*par_noise_intensity)
simple_equation = -par_potential_aplitude * potential()(var_x) + par_const_force - par_gamma * var_v

# runtime options
options = Sde.get_default_options()
options['gpu']['cuda'] = False
# SDE
SDE = Sde(variables, parameters, functions, equation2, options)
SDE.start()

testing

In [82]:
from sympy import init_printing
init_printing()

equation = simple_equation

display(equation)
display(equation.free_symbols)
print(equation.free_symbols)
print(f_potential)
print(variables)

display(equation.evalf())
display(equation.evalf(subs={par_const_force:1, par_gamma:3, var_v:10, var_x:2, par_potential_aplitude:-3}))

{Delta_U, gamma, v, f, x}
U
{x: 0, v: 1, t: 0}


In [57]:
from sympy import Symbol, solve
aa = Symbol("a")
bb = Symbol("b")
cc = Symbol("c")
exp = (aa+bb)*40-(cc-aa)/0.5
print(exp)
print(exp.free_symbols)
print(solve(exp))
print(exp.evalf(subs={aa:0.2, 'b':0.05, 'c':0.7}))

42.0*a + 40*b - 2.0*c
{b, c, a}
[{a: -0.952380952380952*b + 0.0476190476190476*c}]
9.00000000000000


In [66]:
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

a = numpy.random.randn(4,4)       # random numers
a = a.astype(numpy.float32)       # to single precision
a_gpu = cuda.mem_alloc(a.nbytes)  # allocate memory on device
cuda.memcpy_htod(a_gpu, a)        # copy data to allocated memory
# code to execute
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
func = mod.get_function("doublify") # get the function to call
func(a_gpu, block=(4,4,1))          # and call it
a_doubled = numpy.empty_like(a)     # preallocate numpy array
cuda.memcpy_dtoh(a_doubled, a_gpu)  # and put the results there
print (a_doubled)
print (a)

[[ 1.88137901 -1.03420198 -3.05230021 -4.10058928]
 [ 1.13113844 -0.6754241  -0.1893848  -0.34039509]
 [-0.03850011 -0.04408523 -0.59397918  1.5112803 ]
 [ 0.26631567  0.6616053  -1.30357599 -0.57462847]]
[[ 0.9406895  -0.51710099 -1.52615011 -2.05029464]
 [ 0.56556922 -0.33771205 -0.0946924  -0.17019755]
 [-0.01925005 -0.02204261 -0.29698959  0.75564015]
 [ 0.13315783  0.33080265 -0.651788   -0.28731424]]


In [5]:
import functools
import operator
def indices_1D_ND(index, arrs):
    '''
    :param arrs:  array of arrays containing variables/parameters values
                  e.g.: [[0], [0,1,2,3,4,5,6], range(95,105), [2], [1], [0.1, 0.01, 0.001]]
    :param index: if all possible values combinations were produced from respectible subarrays and then flattened into 1D
                  this is gonna be an index of values set in such 1D array
    :returns:     array of indices for accessing desired values in arrs (indices in the same order as arrs)
    :raises ValueError: if index out of range
    '''
    indices_reversed = []
    lens = [len(x) for x in arrs]
    max_index = functools.reduce(operator.mul, lens, 1)
    if index < 0 or index >= max_index:
        raise ValueError('index >= {}'.format(max_index))
    for l in reversed(lens):
        indices_reversed.append(index%l)
        index = index // l
    return(indices_reversed[::-1])

aaa = [[0], [0,1,2,3,4,5,6], range(95,105), [2], [1], [0.1, 0.01, 0.001]]
res = indices_1D_ND(1, aaa)
print(res)

[0, 0, 0, 0, 0, 1]


In [70]:
from mako.template import Template
print(Template("hello ${data}!").render(data="world"))

hello world!
